# LLM + Database Assistant with **Tool-Calling Router**

This notebook shows a production-style pattern where the LLM does **tool selection**, not just Q&A.

### Goal
We want an assistant that:
1. Talks normally for generic questions (`chitchat_tool`).
2. Runs a safe DB lookup for factual business questions (`sql_lookup_tool`).

The model itself selects which `tool` to call and also provides arguments for that tool.
Your code then dispatches to the correct handler from a registry.

This is more extensible than `if/else`, and looks like how real agent systems route capability.

## 1. Setup

We will:
- Initialize OpenAI client (you must set `OPENAI_API_KEY` in your environment).
- Build an in-memory SQLite DB called `invoices`.
- Define a whitelist schema `ALLOWED_SCHEMA`.


In [ ]:
import os, sqlite3, json, re
from typing import List, Dict, Any
from openai import OpenAI

# --- OpenAI client ---
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")
client = OpenAI(api_key=OPENAI_API_KEY)

# --- demo DB ---
conn = sqlite3.connect(":memory:")
cur = conn.cursor()
cur.execute(
    "CREATE TABLE invoices ("
    "id INTEGER PRIMARY KEY,"
    "client_name TEXT,"
    "status TEXT,"
    "amount_in_inr REAL,"
    "invoice_date TEXT"
    ")"
)

rows_seed = [
    (1, "Acme Corp",      "unpaid", 375000, "2025-10-12"),
    (2, "Acme Corp",      "unpaid", 210000, "2025-09-20"),
    (3, "Acme Corp",      "paid",   180000, "2025-09-05"),
    (4, "Skyline Motors", "unpaid", 990000, "2025-10-01"),
    (5, "Skyline Motors", "unpaid", 120000, "2025-08-28")
]
cur.executemany("INSERT INTO invoices VALUES (?, ?, ?, ?, ?)", rows_seed)
conn.commit()

# Whitelisted schema that we are comfortable exposing to the model
ALLOWED_SCHEMA = {
    "invoices": {
        "columns": [
            "id",
            "client_name",
            "status",
            "amount_in_inr",
            "invoice_date"
        ]
    }
}

print("Demo DB initialized. Rows example:", rows_seed[:2])


## 2. Tool-Calling Router

Instead of manual `if intent == ...`, we ask the LLM to **choose a tool** and provide structured arguments. The model returns strict JSON like:

```json
{
  "tool": "sql_lookup_tool",
  "arguments": {
    "client": "Acme",
    "status": "unpaid",
    "amount_gt": 200000,
    "date_after": "2025-09-01",
    "limit": 20
  },
  "explanation": "User is asking for unpaid invoices above a threshold."
}
```

Your runtime then dispatches to the right Python function for that tool.
This is clean and extensible.

In [ ]:
TOOL_ROUTER_PROMPT = """
You are a tool router for an enterprise assistant.

Return STRICT JSON with the shape:
{
  "tool": "<one of: chitchat_tool, sql_lookup_tool>",
  "arguments": { ... },
  "explanation": "short reason for choosing this tool"
}

Rules:
1. Use "chitchat_tool" when the user is asking for general explanations, definitions,
   brainstorming, opinions, strategy, or conceptual answers that do NOT require data from the invoices table.

2. Use "sql_lookup_tool" ONLY if the user is clearly asking for factual business data
   that lives in the invoices table. For example:
   - unpaid invoices
   - outstanding amounts
   - invoices above some threshold
   - date filters like "after 2025-09-01"

3. When you choose "sql_lookup_tool", include arguments:
   - client (string or null)
   - status (string like "unpaid" or null)
   - amount_gt (number or null)
   - date_after (ISO date string like "2025-09-01" or null)
   - limit (integer, default 20)

4. Return ONLY valid JSON. No commentary outside JSON.
"""


def route_to_tool(user_msg: str) -> Dict[str, Any]:
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": TOOL_ROUTER_PROMPT},
            {"role": "user", "content": user_msg}
        ],
        temperature=0.0,
    )
    raw = resp.choices[0].message.content
    return json.loads(raw)


# Demo: routing two messages
for msg in [
    "Explain what embeddings are in simple terms.",
    "Show unpaid invoices for Acme after 2025-09-01 above 200000 rupees."
]:
    print("\nUSER:", msg)
    print("ROUTER->TOOL:", json.dumps(route_to_tool(msg), indent=2))


## 3. SQL Planner (for `sql_lookup_tool`)

If the chosen tool is `sql_lookup_tool`, we call a planner model to draft SQL.
We give it:
- Only the safe, whitelisted schema
- The structured arguments from the router

We force rules:
- Only `SELECT`
- No destructive commands
- No `SELECT *`
- Include `LIMIT`
- Include human-readable `purpose` for audit


In [ ]:
SQL_PLANNER_PROMPT = """
You are a SQL planner for a reporting assistant.

Return STRICT JSON:
{
  "sql": "...",
  "purpose": "short english reason for the query"
}

Rules:
1. ONLY use this allowed schema:
   invoices(id, client_name, status, amount_in_inr, invoice_date)

2. ONLY generate SELECT statements.
3. NEVER use INSERT/UPDATE/DELETE/ALTER/DROP/TRUNCATE/CREATE.
4. Do NOT use SELECT *.
5. Include ORDER BY invoice_date DESC if a date filter is relevant.
6. Include LIMIT (e.g. LIMIT 20) unless the user explicitly said "all".
"""


def plan_sql(arguments: Dict[str, Any], schema: Dict[str, Any]) -> Dict[str, Any]:
    payload = {
        "allowed_schema": schema,
        "requested_filter": arguments
    }

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": SQL_PLANNER_PROMPT},
            {"role": "user", "content": json.dumps(payload, indent=2)}
        ],
        temperature=0.0,
    )

    raw = resp.choices[0].message.content
    return json.loads(raw)


# quick dry run using a pretend router output
fake_args = {
    "client": "Acme",
    "status": "unpaid",
    "amount_gt": 200000,
    "date_after": "2025-09-01",
    "limit": 20
}
sql_plan = plan_sql(fake_args, ALLOWED_SCHEMA)
print("SQL_PLAN:", json.dumps(sql_plan, indent=2))


## 4. SQL Validator (pure code, no model)

Before executing any model-generated SQL, we validate:
- starts with `SELECT`
- no forbidden keywords (DROP, UPDATE, etc.)
- only allowed tables/columns from `ALLOWED_SCHEMA`
- has LIMIT (or at least warn)

If validation fails, we DO NOT run the SQL.


In [ ]:
def validate_sql(sql: str, allowed_schema: Dict[str, Any]) -> bool:
    s = sql.lower()

    # Must begin with SELECT
    if not s.strip().startswith("select "):
        print("❌ not a SELECT")
        return False

    # Block destructive keywords
    forbidden = [" delete ", " update ", " insert ", " alter ", " drop ", " truncate ", " create "]
    for token in forbidden:
        if token in s:
            print("❌ forbidden keyword:", token.strip())
            return False

    # Extract table names after FROM / JOIN
    tables = re.findall(r"(?:from|join)\s+([a-zA-Z_][a-zA-Z0-9_]*)", s)
    for t in tables:
        if t not in allowed_schema:
            print("❌ table not allowed:", t)
            return False

    # Extract columns between SELECT ... FROM
    m = re.search(r"select\s+(.*?)\s+from", s, re.DOTALL)
    if m:
        cols_part = m.group(1)
        cols = [c.strip() for c in cols_part.split(",")]
        for c in cols:
            base = c.split()[0]  # drop aliases if present
            if "." in base:
                tname, cname = base.split(".", 1)
            else:
                tname, cname = None, base

            if tname and tname in allowed_schema:
                if cname not in allowed_schema[tname]["columns"]:
                    print(f"❌ column {tname}.{cname} not allowed")
                    return False

    # LIMIT check (soft enforcement for demo)
    if " limit " not in s:
        print("⚠️ No LIMIT found (allowing for demo).")

    print("SQL validated ✅")
    return True


_ = validate_sql(sql_plan["sql"], ALLOWED_SCHEMA)


## 5. Execute Safely and Synthesize Answer

Now we:
1. Run the validated SQL on a **read-only** connection.
2. Ask an Answer Synthesizer model to turn raw rows into a clean business answer.

User should not see raw SQL unless explicitly requested. We answer like a helpful analyst, not like a database dump.

In [ ]:
def run_readonly(sql: str) -> List[Dict[str, Any]]:
    c = conn.cursor()
    c.execute(sql)
    cols = [d[0] for d in c.description]
    out = []
    for row in c.fetchall():
        out.append(dict(zip(cols, row)))
    return out


ANSWER_SYNTH_PROMPT = """
You are a business assistant.
Given:
1. original user question (the intent of the query)
2. executed SQL
3. rows returned (JSON array of objects)

Return a short, confident, human-readable answer.
Do not invent data that is not present.
If there are many rows, summarize count and highlights.
"""


def synthesize_answer(user_question: str, sql: str, rows: List[Dict[str, Any]]) -> str:
    r = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": ANSWER_SYNTH_PROMPT},
            {"role": "user", "content": json.dumps({
                "user_question": user_question,
                "sql_executed": sql,
                "rows": rows
            }, indent=2)}
        ],
        temperature=0.3
    )
    return r.choices[0].message.content


demo_rows = run_readonly(sql_plan["sql"])
print("DB ROWS:", json.dumps(demo_rows, indent=2))

print("\nFINAL ANSWER:")
print(synthesize_answer(
    "Show unpaid invoices for Acme after 2025-09-01 above 200000 rupees.",
    sql_plan["sql"],
    demo_rows
))


## 6. Tool Registry and Dispatcher

Now we wire the whole thing together.

We keep a registry:
```python
TOOLS = {
    "chitchat_tool": run_chitchat_tool,
    "sql_lookup_tool": run_sql_lookup_tool
}
```

The router LLM returns `{tool, arguments}`. We look up the matching Python function and execute it.

This gives you:
- Extensibility (add a new tool without rewriting router code)
- Auditability (we know exactly which tool ran)
- Security hooks (you can add authorization before running any tool)


In [ ]:
def run_chitchat_tool(arguments: Dict[str, Any]) -> str:
    # arguments may pass the cleaned question
    user_q = arguments.get("question", "") or arguments.get("raw_user_input", "")
    if not user_q:
        user_q = "Answer helpfully."

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful, precise assistant."},
            {"role": "user", "content": user_q}
        ],
        temperature=0.4
    )
    return resp.choices[0].message.content


def run_sql_lookup_tool(arguments: Dict[str, Any]) -> str:
    """
    arguments example:
    {
      "client": "Acme",
      "status": "unpaid",
      "amount_gt": 200000,
      "date_after": "2025-09-01",
      "limit": 20
    }
    Steps:
    1. Ask planner to draft SQL
    2. Validate SQL
    3. Execute on read-only DB
    4. Summarize rows back to the user
    """
    # 1. plan SQL
    plan = plan_sql(arguments, ALLOWED_SCHEMA)
    sql = plan["sql"]

    # 2. validate
    if not validate_sql(sql, ALLOWED_SCHEMA):
        return "I understand the request but I am not allowed to run that query safely."

    # 3. execute
    result_rows = run_readonly(sql)

    # 4. synthesize
    # we use the arguments dict as a stand-in for 'user question' context
    user_q = json.dumps(arguments)
    return synthesize_answer(user_q, sql, result_rows)


TOOLS = {
    "chitchat_tool": run_chitchat_tool,
    "sql_lookup_tool": run_sql_lookup_tool
}


def answer_user_message(user_message: str) -> str:
    # Step 1: LLM chooses a tool + arguments
    decision = route_to_tool(user_message)
    tool_name = decision.get("tool", "chitchat_tool")
    args = decision.get("arguments", {})
    tool_fn = TOOLS.get(tool_name, run_chitchat_tool)

    # (Optional) authorization layer would go here before executing tool_fn
    return tool_fn(args)


print("DEMO SQL STYLE QUESTION:")
print(answer_user_message("Show unpaid invoices for Acme after 2025-09-01 above 200000 rupees."))

print("\nDEMO CHITCHAT QUESTION:")
print(answer_user_message("Explain embeddings like I am new to machine learning."))


## 7. Recap — Why This Is Production-Grade

1. The model does NOT just answer directly. It chooses a `tool`.
2. Each tool is a governed capability. You can review, test, and lock it down.
3. The SQL path is STILL protected:
   - SQL planner creates a candidate query
   - SQL validator enforces safety rules
   - Execution happens on read-only data
   - The final answer is synthesized in plain human language

4. Adding a new enterprise skill is easy:
   - Write a new tool function
   - Document that tool in the router prompt
   - Add to `TOOLS`
   No changes to the dispatcher logic.

This is how you move from 'cute demo bot' to 'responsible AI assistant in front of leadership and audit.'